In [1]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import pandas as pd
import re]
import 

[nltk_data] Downloading package punkt to /Users/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [145]:
# reading in the data
train_data = pd.read_csv('WikiQA-train.tsv', sep='\t')
test_data = pd.read_csv('WikiQA-test.tsv', sep='\t')

Extract the unique questions from the train and test data frames, including the documentID and the DocumentTitle

In [146]:
def get_questions_documenttag(data):
    qd = data[['Question', 'QuestionID', 'DocumentID','DocumentTitle']].drop_duplicates()
    return qd
train_question_doctag = get_questions_documenttag(train_data)
test_question_doctag = get_questions_documenttag(test_data)

In [147]:
# get unique questions
train_questions = train_question_doctag['Question']
test_questions = test_question_doctag['Question']

In [148]:
# get the unique document ids
train_docid = train_question_doctag['DocumentID']
test_docid = test_question_doctag['DocumentID']

Extract the answers to those questions.

In [149]:
def get_answers(data, questions, documentids): 
    answers = [] # list of answers
    for q in range(len(questions)):
        question = questions.iloc[q]
        doc_id = documentids.iloc[q] # add the document id
        df = data[data['Question'] == question]
        index = df.loc[df['Label'] == 1]['Sentence'].index.values
        if len(index) == 0: # if no answer found
            answers.append([question, doc_id, 'No answer'])
        else: # if 1 answer found
            answers.append([question, doc_id, df.loc[index[0], "Sentence"]])
    return answers

train_answers = pd.DataFrame(get_answers(train_data, train_questions, train_docid))
test_answers = pd.DataFrame(get_answers(test_data, test_questions, test_docid))

The above get_answers returns train_answers and test_answers which, gives us in the following columns
- Question
- Related Document ID
- Answer (if no answer to that question, return no answer)

In [150]:
def get_documents(data, questions, documentids): # (done by Finn, tweaked by Dan)
    documents = []
    for q in range(len(questions)):
        question = questions.iloc[q]
        doc_id = documentids.iloc[q] # add the document id
        df = data[data['Question'] == question]
        sentences = df['Sentence'].tolist()
        for i in range(0, len(sentences) - 1):
            sentences[i] = sentences[i] + ' '
        documents.append([doc_id,''.join(sentences)])
    return documents

train_documents = pd.DataFrame(get_documents(train_data, train_questions, train_docid)) # return the individual document in list
test_documents = pd.DataFrame(get_documents(test_data, test_questions, test_docid)) # return the individual document in list

The above train_documents and test_documents called from the get_documents gives us in the following columns
- Document ID
- Full Document

In [151]:
# renaming all the columns for more standardised access
train_answers.columns = ['Question','DocumentID','Answer']
test_answers.columns = ['Question','DocumentID','Answer']
train_documents.columns = ['DocumentID','Document']
test_documents.columns = ['DocumentID','Document']

In [152]:
# result is 2117, 2117, 630, 630

len(train_answers),len(train_documents), len(test_answers),len(test_documents)

(2117, 2117, 630, 630)

**Prior to tagging, we should maybe clean the document and answers first:** (stopped here)

Maybe? 
- lowercase (might lose context, but we can use on questions)
- removing any punctuation or weird symbols (do)
- removal of stop words? (probably not)

Make sure that the pre-processing is standardised to be the same throughout doc and ans.

In [153]:
def preprocess_lower(text):
    # Lowercase the text for question, answer and documents
    text = text.lower()
    return text

train_answers[['Question', 'Answer']] = train_answers[['Question', 'Answer']].applymap(preprocess_lower)
train_documents['Document'] = train_documents['Document'].apply(preprocess_lower)
test_answers[['Question', 'Answer']] = test_answers[['Question', 'Answer']].applymap(preprocess_lower)
test_documents['Document'] = test_documents['Document'].apply(preprocess_lower)

In [154]:
train_documents

,DocumentID,Document
0,D1,a partly submerged glacier cave on perito more...
1,D2,"in physics , circular motion is a movement of ..."
2,D5,apollo creed is a fictional character from the...
3,D6,"in the united states, the title of federal jud..."
4,D7,the beretta 21a bobcat is a small pocket-sized...
...,...,...
2112,D2805,blue mountain state is an american comedy seri...
2113,D2806,"apple inc., formerly apple computer, inc., is ..."
2114,D2807,section 8 housing in the south bronx section 8...
2115,D2808,restaurants categorized by type and informatio...


In [155]:
def labelling(documents, answers):
    tagged_documents = []
    for q in range(len(answers)):
        tagged_document = []
        qn = answers['Question'].loc[q]
        doc_id = answers['DocumentID'].loc[q]
        content = documents.loc[documents['DocumentID'] == doc_id,'Document'].values[0]
        answer = answers['Answer'].loc[q]

        if answer == 'no answer':
            tokens = word_tokenize(content)
            for j in range(len(tokens)):
                tagged_document.append('N') # none 
        else:
            parts = content.partition(answer)
            for j in range(len(parts)):
                tokens = word_tokenize(parts[j])
                if j == 1:
                    tagged_document.append('S') # start of answer
                    for k in range(len(tokens) - 2):
                        tagged_document.append('I') # inside of answer
                    tagged_document.append('E') # end of answer
                else:
                    for k in range(len(tokens)):
                        tagged_document.append('N') # outside answer
        tagged_documents.append(tagged_document)
    return(tagged_documents)

train_doc_ans_labels = labelling(train_documents, train_answers)
test_doc_ans_labels = labelling(test_documents, test_answers)

In [156]:
# check if tags are good
def testing_tokens(ind, labels, documents, answers):
    for i,j in zip(labels[ind],word_tokenize(documents['Document'][ind])):
        print([i,j])
    print(answers['Answer'][ind])
testing_tokens(100 , train_doc_ans_labels, train_documents, train_answers)

['N', 'the']
['N', 'big']
['N', 'ten']
['N', 'conference']
['N', ',']
['N', 'formerly']
['N', 'western']
['N', 'conference']
['N', 'and']
['N', 'big']
['N', 'nine']
['N', 'conference']
['N', ',']
['N', 'is']
['N', 'the']
['N', 'oldest']
['N', 'division']
['N', 'i']
['N', 'college']
['N', 'athletic']
['N', 'conference']
['N', 'in']
['N', 'the']
['N', 'united']
['N', 'states']
['N', '.']
['S', 'its']
['I', 'twelve']
['I', 'member']
['I', 'institutions']
['I', '(']
['I', 'which']
['I', 'are']
['I', 'primarily']
['I', 'flagship']
['I', 'research']
['I', 'universities']
['I', 'in']
['I', 'their']
['I', 'respective']
['I', 'states']
['I', ',']
['I', 'well-regarded']
['I', 'academically']
['I', ',']
['I', 'and']
['I', 'with']
['I', 'relatively']
['I', 'large']
['I', 'student']
['I', 'enrollment']
['I', ')']
['I', 'are']
['I', 'located']
['I', 'primarily']
['I', 'in']
['I', 'the']
['I', 'midwest']
['I', ',']
['I', 'stretching']
['I', 'from']
['I', 'nebraska']
['I', 'in']
['I', 'the']
['I', 'we

Cleaned Documents: train and test

train_answers - contains the ['Question','DocumentID','Answer'] 

train_documents - contains the ['DocumentID','Document']

train_doc_ans_labels - contains a list of list of answer tags for each document, 

In [157]:
# To prepare the document for word embeddings:
train_doc_ques = pd.DataFrame({'Document': train_documents['Document'],
                               'Question': train_answers['Question']})
test_doc_ques = pd.DataFrame({'Document': test_documents['Document'],
                               'Question': test_answers['Question']})

### Word Embeddings

To use the CBOW model, we need the data in sentences. Extract this from the original dataset, don't use sent_tokenise, will mess with some of the fullstops, we want to maintain structure from above

In [158]:
def word_tokens(data):
    sentence_list = []
    for i in range(len(data)):
        sentence_list.append(word_tokenize(data[i]))
    return(sentence_list)
train_doc_list = word_tokens(train_doc_ques['Document'])
train_ques_list = word_tokens(train_doc_ques['Question'])
test_doc_list = word_tokens(test_doc_ques['Document'])
test_ques_list = word_tokens(test_doc_ques['Question'])

In [159]:
combined_text = train_doc_list + train_ques_list + test_doc_list + test_ques_list

In [44]:
from gensim.models import Word2Vec
wc_cbow_model = Word2Vec(sentences=combined_text, vector_size=100, window=5, min_count=1, workers=2, epochs=30)
wc_cbow_model.save("cbow.model")

To implement QA

1. Word Embeddings, using CBOW
2. Feature Extraction 1 - POS tags
3. Feature Extraction 2 - TF-IDF 
4. Feature Extraction 3 - NER

In [160]:
def get_word_embeddings(doc):
    tokenized_doc = word_tokenize(doc)
    embeddings = [wc_cbow_model.wv[word] for word in tokenized_doc]
    return embeddings

train_doc_ques['Doc_Embeddings'] = train_doc_ques['Document'].apply(get_word_embeddings)
train_doc_ques['Q_Embeddings'] = train_doc_ques['Question'].apply(get_word_embeddings)
test_doc_ques['Doc_Embeddings'] = test_doc_ques['Document'].apply(get_word_embeddings)
test_doc_ques['Q_Embeddings'] = test_doc_ques['Question'].apply(get_word_embeddings)

In [161]:
train_doc_ques['Doc_Tokens'] = train_doc_ques['Document'].apply(word_tokenize)
train_doc_ques['Q_Tokens'] =  train_doc_ques['Question'].apply(word_tokenize)
test_doc_ques['Doc_Tokens'] = test_doc_ques['Document'].apply(word_tokenize)
test_doc_ques['Q_Tokens'] = test_doc_ques['Question'].apply(word_tokenize)

In [162]:
def check_count(doc):
    count = 0
    for i in range(len(doc)):
        if len(doc['Doc_Embeddings'][i]) != len(doc['Doc_Tokens'][i]):
            count += 1
        elif len(doc['Q_Embeddings'][i]) != len(doc['Q_Tokens'][i]):
            count += 1
        else:
            continue
    return(count)
        
check_count(train_doc_ques) # looks good

0

Note, need to convert the POS tags, NER tags into embeddings. After this, pad the questions and answers to the max question/document length in the combined training and test set.

### PoS Tagging

In [163]:
# Apply the pos tags to the tokens 
from nltk.tag import pos_tag
# download the dependency and resource as required
nltk.download('averaged_perceptron_tagger')

train_doc_ques['Doc_POS'] = train_doc_ques['Doc_Tokens'].apply(pos_tag)
train_doc_ques['Q_POS'] =  train_doc_ques['Q_Tokens'].apply(pos_tag)
test_doc_ques['Doc_POS'] = test_doc_ques['Doc_Tokens'].apply(pos_tag)
test_doc_ques['Q_POS'] = test_doc_ques['Q_Tokens'].apply(pos_tag)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/daniel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [164]:
# checking the POS tags: # looks ok
test_doc_ques['Q_POS'][0]

[('how', 'WRB'),
 ('african', 'JJ'),
 ('americans', 'NNS'),
 ('were', 'VBD'),
 ('immigrated', 'VBN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('us', 'PRP')]

In [184]:
# Extract all unique POS Tags
all_pos_tags = train_doc_ques['Doc_POS'].tolist() + test_doc_ques['Doc_POS'].tolist() + train_doc_ques['Q_POS'].tolist() + test_doc_ques['Q_POS'].tolist()

def get_unique_pos(data):
    pos_tags = set()
    for item in data:
        for _,pos_tag in item:
            pos_tags.add(pos_tag)

    pos_tag_index = {tag: i for i, tag in enumerate(sorted(pos_tags))}
    return pos_tag_index

pos_iden = get_unique_pos(all_pos_tags) # list of tags
pos_iden

{'#': 0,
 '$': 1,
 "''": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '.': 6,
 ':': 7,
 'CC': 8,
 'CD': 9,
 'DT': 10,
 'EX': 11,
 'FW': 12,
 'IN': 13,
 'JJ': 14,
 'JJR': 15,
 'JJS': 16,
 'MD': 17,
 'NN': 18,
 'NNP': 19,
 'NNPS': 20,
 'NNS': 21,
 'PDT': 22,
 'POS': 23,
 'PRP': 24,
 'PRP$': 25,
 'RB': 26,
 'RBR': 27,
 'RBS': 28,
 'RP': 29,
 'SYM': 30,
 'TO': 31,
 'UH': 32,
 'VB': 33,
 'VBD': 34,
 'VBG': 35,
 'VBN': 36,
 'VBP': 37,
 'VBZ': 38,
 'WDT': 39,
 'WP': 40,
 'WP$': 41,
 'WRB': 42,
 '``': 43}

### NER Tagging

### Steps to run this:

- pip install spacy 
- python -m spacy download en_core_web_sm

If loaded for the first time, restart kernel

In [90]:
# nltk using Spacy
# pip install -U spacy
# python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm

# loading pre-trained model of NER
nlp = en_core_web_sm.load()

In [121]:
def ner_tagging(texts):
    tagged_texts = []
    for text in texts:
        doc = spacy.tokens.Doc(nlp.vocab, words=text)
        nlp.get_pipe("ner")(doc)
        tagged_texts.append([(token.text, token.ent_type_) for token in doc])
    return tagged_texts

In [182]:
# Will take a while...
train_doc_ques['Doc_NER'] = ner_tagging(train_doc_ques['Doc_Tokens'])
train_doc_ques['Q_NER'] = ner_tagging(train_doc_ques['Q_Tokens'])
test_doc_ques['Doc_NER'] = ner_tagging(test_doc_ques['Doc_Tokens'])
test_doc_ques['Q_NER'] = ner_tagging(test_doc_ques['Q_Tokens'])

In [185]:
# Similar approach to the POS

# Extract all unique POS Tags
all_ner_tags = train_doc_ques['Doc_NER'].tolist() + test_doc_ques['Doc_NER'].tolist() + train_doc_ques['Q_NER'].tolist() + test_doc_ques['Q_NER'].tolist()

def get_unique_ner(data):
    ner_tags = set()
    for item in data:
        for _,ner_tag in item:
            ner_tags.add(ner_tag)

    ner_tag_index = {tag: i for i, tag in enumerate(sorted(ner_tags))}
    return ner_tag_index

ner_iden = get_unique_pos(all_ner_tags) # list of tags
ner_iden

{'': 0,
 'CARDINAL': 1,
 'DATE': 2,
 'EVENT': 3,
 'FAC': 4,
 'GPE': 5,
 'LANGUAGE': 6,
 'LAW': 7,
 'LOC': 8,
 'MONEY': 9,
 'NORP': 10,
 'ORDINAL': 11,
 'ORG': 12,
 'PERCENT': 13,
 'PERSON': 14,
 'PRODUCT': 15,
 'QUANTITY': 16,
 'TIME': 17,
 'WORK_OF_ART': 18}

In [186]:
# Stopped here